In [1]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
from pathlib import Path
import json
import os

from playwright.async_api import async_playwright

# --- Scraper settings ---
USER = "financialjuice"          # handle to scrape
HOURS_BACK = 24                  # last N hours
OUT_TXT = "financialjuice_last_hours.txt"
OUTPUT_TZ = "Europe/Zurich"      # Geneva time
MAX_SCROLLS = 80                 # increase for more tweets
SCROLL_WAIT_MS = 1600            # increase if loading is slow (e.g., 2000+)
COOKIES_FILE = "x_cookies.json"  # optional: export your x.com cookies to this file

UA = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/124.0 Safari/537.36"
)

In [ ]:
# Run this ONCE if anonymous scraping yields 0 tweets.
# It opens a real browser window. Log in to X, then press Enter in the notebook.
async def login_and_save_cookies():
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=False, args=["--disable-blink-features=AutomationControlled"]
        )
        ctx = await browser.new_context(user_agent=UA, viewport={"width":1200,"height":900})
        page = await ctx.new_page()
        await page.goto("https://x.com/i/flow/login", wait_until="domcontentloaded", timeout=120_000)
        print("A browser window opened. Log in to X normally. When your timeline is visible, return here and press Enter.")
        input()
        cookies = await ctx.cookies()
        with open(COOKIES_FILE, "w", encoding="utf-8") as f:
            json.dump(cookies, f, ensure_ascii=False, indent=2)
        await browser.close()
        print(f"Cookies saved to {COOKIES_FILE}")

# Usage (uncomment to run once):
#await login_and_save_cookies()

A browser window opened. Log in to X normally. When your timeline is visible, return here and press Enter.
Cookies saved to x_cookies.json


In [7]:
async def scrape_last_hours(user: str, hours_back: int = 24):
    cutoff = datetime.now(timezone.utc) - timedelta(hours=hours_back)
    tz = ZoneInfo(OUTPUT_TZ)
    rows, seen = [], set()

    async with async_playwright() as p:
        # Chromium is a bit more reliable on X; switch to firefox if you prefer
        browser = await p.chromium.launch(
            headless=True, args=["--disable-blink-features=AutomationControlled"]
        )
        ctx = await browser.new_context(user_agent=UA, viewport={"width":1280,"height":2000})

        # Reuse your logged-in session if available (recommended)
        if Path(COOKIES_FILE).exists():
            try:
                with open(COOKIES_FILE, "r", encoding="utf-8") as f:
                    await ctx.add_cookies(json.load(f))
            except Exception:
                pass

        page = await ctx.new_page()
        await page.goto(f"https://x.com/{user}", wait_until="domcontentloaded", timeout=90_000)

        # Try to dismiss consent/overlays (best effort)
        for label in ("Accept", "I agree", "Allow all"):
            try:
                await page.get_by_role("button", name=label).click(timeout=1500)
            except Exception:
                pass

        # Scroll & collect
        for _ in range(MAX_SCROLLS):
            arts = await page.query_selector_all('article[data-testid="tweet"]')
            for a in arts:
                link_el = await a.query_selector('a[role="link"][href*="/status/"]')
                if not link_el:
                    continue
                href = await link_el.get_attribute("href")
                if not href or href in seen:
                    continue

                t_el = await a.query_selector("time")
                if not t_el:
                    continue
                dt_str = await t_el.get_attribute("datetime")
                if not dt_str:
                    continue

                # parse ISO timestamp (UTC), filter by cutoff
                dt = datetime.fromisoformat(dt_str.replace("Z", "+00:00"))
                if dt < cutoff:
                    continue

                parts = await a.query_selector_all('[data-testid="tweetText"]')
                text = "\n".join([await n.inner_text() for n in parts]).strip() if parts else ""

                rows.append({"time": dt.astimezone(tz), "text": text})
                seen.add(href)

            # scroll for more
            await page.mouse.wheel(0, 20000)
            await page.wait_for_timeout(SCROLL_WAIT_MS)

        await browser.close()

    # sort newest → oldest & dedupe by (time, text)
    out_map = {(r["time"].isoformat(), r["text"]): r for r in rows}
    out = list(out_map.values())
    out.sort(key=lambda r: r["time"], reverse=True)
    return out

In [8]:
rows = await scrape_last_hours(USER, HOURS_BACK)
print(f"Collected {len(rows)} tweets in last {HOURS_BACK}h")

with open(OUT_TXT, "w", encoding="utf-8") as f:
    for r in rows:
        f.write(f"{r['time'].strftime('%Y-%m-%d %H:%M:%S %Z')} | {r['text']}\n\n")

print(f"Saved -> {OUT_TXT}")

Collected 36 tweets in last 24h
Saved -> financialjuice_last_hours.txt


In [10]:
import google.generativeai as genai
from typing import List, Tuple
import pathlib

# Set your key in the environment before running (e.g., in a terminal: export GOOGLE_API_KEY="...")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("Missing GOOGLE_API_KEY environment variable.")
genai.configure(api_key=GOOGLE_API_KEY)

MODEL_NAME = "gemini-1.5-flash"   # or "gemini-1.5-pro"

RuntimeError: Missing GOOGLE_API_KEY environment variable.

In [ ]:
def load_tweets(path: str = "financialjuice_last_hours.txt") -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()

def chunk_text(s: str, max_chars: int = 15000) -> List[str]:
    """Greedy chunking by blank-line separated entries, staying under max_chars per chunk."""
    s = s.replace("\r\n", "\n")
    chunks, cur, total = [], [], 0
    for block in s.split("\n\n"):
        block += "\n\n"
        if cur and total + len(block) > max_chars:
            chunks.append("".join(cur))
            cur, total = [], 0
        cur.append(block)
        total += len(block)
    if cur:
        chunks.append("".join(cur))
    return chunks

def summarize_chunks(chunks: List[str], custom_prompt: str) -> List[str]:
    model = genai.GenerativeModel(MODEL_NAME)
    system = "Follow the user’s instructions exactly. Do not add extra sections beyond what they ask."
    outputs = []
    for i, ch in enumerate(chunks, 1):
        prompt = (
            custom_prompt
            + f"\n\nCHUNK {i}/{len(chunks)} — TWEETS START\n<<<\n{ch}\n>>>\n"
            + "Return a concise markdown summary (headings + bullet points)."
        )
        resp = model.generate_content([system, prompt])
        outputs.append((resp.text or "").strip())
    return outputs

def final_synthesis(per_chunk: List[str], custom_prompt: str) -> str:
    model = genai.GenerativeModel(MODEL_NAME)
    joined = "\n\n--- CHUNK SPLIT ---\n\n".join(per_chunk)
    prompt = (
        custom_prompt
        + "\n\nYou are given partial summaries of tweet batches. "
          "Merge them into ONE well-structured Markdown document following the exact instructions above.\n\n"
          "PARTIAL SUMMARIES START\n<<<\n" + joined + "\n>>>\n"
          "Return ONLY the final markdown."
    )
    resp = model.generate_content(prompt)
    return (resp.text or "").strip()

def summarize_tweets_to_md(
    tweets_path: str = "financialjuice_last_hours.txt",
    output_md: str = "summary.md",
    custom_prompt: str = "",
    max_chars_per_chunk: int = 15000,
) -> Tuple[str, int, int]:
    raw = load_tweets(tweets_path)
    if not raw:
        raise ValueError(f"Tweet file is empty: {tweets_path}")

    chunks = chunk_text(raw, max_chars=max_chars_per_chunk)
    per_chunk = summarize_chunks(chunks, custom_prompt)
    md = final_synthesis(per_chunk, custom_prompt)

    pathlib.Path(output_md).write_text(md, encoding="utf-8")
    return output_md, len(raw), len(chunks)

In [ ]:
custom_prompt = (
    "You are drafting a clear, trader‑ready summary. Be concise. "
    "Group by **Macro Area** (in this order: Europe, UK, North America, Asia) and then **Countries** "
    "Avoid repetition. Prefer bullet points over prose; keep each bullet short and factual. "
    "If there is a news that is relevant for europe in general, or for many countries, place it under the **Europe**, without creating a new section. "
    "Use the timestamps to resolve conflicts: if a newer tweet updates an older one, only keep the updated fact. "
    "If a data release includes estimates/priors vs actuals, show the comparison succinctly. "
    "Output must be valid Markdown."
)

out_file, n_chars, n_chunks = summarize_tweets_to_md(
    tweets_path=OUT_TXT,
    output_md="summary.md",
    custom_prompt=custom_prompt,
    max_chars_per_chunk=15000,
)

print(f"Done. Input chars: {n_chars}, chunks: {n_chunks}")
print(f"Saved -> {out_file}")